Import all packages here

In [2]:
import tensorflow as tf
from transformers import Wav2Vec2Processor, TFWav2Vec2Model

import librosa
import soundfile


import scipy as sy
import pydub as pyd



import jiwer as jw





import pandas as pd 
import numpy as np 

RuntimeError: Failed to import transformers.models.wav2vec2.modeling_tf_wav2vec2 because of the following error (look up to see its traceback):
Your currently installed version of Keras is Keras 3, but this is not yet supported in Transformers. Please install the backwards-compatible tf-keras package with `pip install tf-keras`.

Mounting Gdrive for the datasets

Setting up the Dataset

Data Pre-Processing

Training the Model

Training Results

Visualization of Results using MatPlotLib